The channels I watched LOL are: #Tubbo #yamikazexz #riotgames #nightblue3 #iwilldominate #sneakylol #thebausffs #tfblade #loltyler1 #shiphtur #tarzaned #jankos #ratirl #drututt #katevolved #wingsofdeath #trick2g #karasmai #ipav999 #anniebot #boxbox #corejj #lol_selfmade # nisqyy #ikeepittaco #gamergirl #tobiasfate

The channels I watched PUBG are: #Tubbo #danucd #tgltn #ibiza #hambinooo #break #halifax #pubg_andymh5 #grizz #alisa #summit1g #chocotaco #feyd #fuzzfaze49 #ashek #gagod #chad #taryn #jowybear #shrimzy

https://www.twitch.tv/tubbo

The tutorial: https://www.learndatasci.com/tutorials/how-stream-text-data-twitch-sockets-python/

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_project_folder = 'Data-X: GGWP Toxic Behavior Public Data'

In [ ]:
%cd drive/My Drive/{my_project_folder}/scraping

In [49]:
server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'aki_niki'  # your username
token = '' # your token 
channel = '#tobiasfate'

In [50]:
# instantiate a socket
import socket
sock = socket.socket()

In [51]:
sock.connect((server, port))

In [52]:
sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

17

In [53]:
resp = sock.recv(2048).decode('utf-8')
resp

':tmi.twitch.tv 001 aki_niki :Welcome, GLHF!\r\n:tmi.twitch.tv 002 aki_niki :Your host is tmi.twitch.tv\r\n:tmi.twitch.tv 003 aki_niki :This server is rather new\r\n:tmi.twitch.tv 004 aki_niki :-\r\n:tmi.twitch.tv 375 aki_niki :-\r\n:tmi.twitch.tv 372 aki_niki :You are in a maze of twisty passages, all alike.\r\n:tmi.twitch.tv 376 aki_niki :>\r\n:aki_niki!aki_niki@aki_niki.tmi.twitch.tv JOIN #tobiasfate\r\n:aki_niki.tmi.twitch.tv 353 aki_niki = #tobiasfate :aki_niki\r\n:aki_niki.tmi.twitch.tv 366 aki_niki #tobiasfate :End of /NAMES list\r\n'

###### Write into files

In [54]:
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler('chat.log', encoding='utf-8')])

In [55]:
logging.info(resp)

In [56]:
# This process is keep collecting data. Interrupt it to go to the next step

from emoji import demojize

while True:
    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))
        
# run in terminal: tail -f chat.log

KeyboardInterrupt: 

In [57]:
# sample message
msg = "2020-11-24_16:54:53 — :flam1ngoat!flam1ngoat@flam1ngoat.tmi.twitch.tv PRIVMSG #tobiasfate :I think"
#'2018-12-10_11:26:40 — :spappygram!spappygram@spappygram.tmi.twitch.tv PRIVMSG #ninja :Chat, let Ninja play solos'

In [58]:
# date: split it off and parse it
from datetime import datetime

time_logged = msg.split()[0].strip()

time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

time_logged

datetime.datetime(2020, 11, 24, 16, 54, 53)

In [59]:
username_message = msg.split('—')[1:]
username_message = '—'.join(username_message).strip()

username_message

':flam1ngoat!flam1ngoat@flam1ngoat.tmi.twitch.tv PRIVMSG #tobiasfate :I think'

In [60]:
import re

username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message).groups()
#  (.*) — will capture part of the string

print(f"Channel: {channel} \nUsername: {username} \nMessage: {message}")

Channel: tobiasfate 
Username: flam1ngoat 
Message: I think


In [61]:
import pandas as pd

def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n\n')
        
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)
        
    
df = get_chat_dataframe('chat.log')

In [62]:
df.set_index('dt', inplace=True)

print(df.shape)

df

(106545, 3)


,channel,username,message
dt,,,
2020-11-19 10:28:01,tfblade,hunter43k,yeah yeah insta 100 %
2020-11-19 10:28:04,tfblade,feudalrat,DOUBTERS PepeLaugh TeaTime DOUBTERS PepeLaugh ...
2020-11-19 10:28:06,tfblade,mylovesachan,SadChamp -2k on bet
2020-11-19 10:28:07,tfblade,exoooox3,WeirdChamp abuser
2020-11-19 10:28:08,tfblade,slimjim3010,I THOUGHT TRYND WAS DEAD
...,...,...,...
2020-11-25 21:46:21,tobiasfate,forgottenvoidx,I think he is fine losing lane to GP.. he just...
2020-11-25 21:46:22,tobiasfate,9kakosgeros6,doesn't*
2020-11-25 21:46:22,tobiasfate,airfoil,ap sion coming in hot


In [63]:
sock.close()

# Conver to CSV

In [ ]:
# Get only the message column as only that is required for the model prediction
df2 = df['message'].to_frame() 
df2 = df2.drop_duplicates() # drop duplicates

In [64]:
game_platform = "pubg_twitch"
df2.to_csv('../data/scraped/' + game_platform + '.csv', encoding='utf-8')